In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [2]:
from google.cloud import aiplatform as vertex_ai
import os
import time

BUCKET = 'jt-merlin-scaling'
BUCKET_URI = 'gs://jt-merlin-scaling'

vertex_ai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
REPO_DOCKER_PATH_PREFIX = 'src'
SERVING_SUB_DIR = 'serving'
SERVING_APPLICATION_DIR = 'app'
SERVING_DOCKERNAME = 'merlin-retriever'
LOCAL_WORKFLOW_DIR = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/local_workflow'

REPOSITORY='sp-mm-cpr'

In [8]:
! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$LOCATION

Create request issued for: [sp-mm-cpr]
Waiting for operation [projects/hybrid-vertex/locations/us-central1/operations/
554b6010-4e47-4c34-9654-9ffa07f12458] to complete...done.                      
Created repository [sp-mm-cpr].


In [9]:
! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [10]:
# # Make the training subfolder
# ! rm -rf {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
# ! mkdir {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
# ! mkdir {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}
# ! touch {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/{SERVING_APPLICATION_DIR}/__init__.py

# !rm -rf $LOCAL_WORKFLOW_DIR
# !mkdir $LOCAL_WORKFLOW_DIR

# ! rm -rf app
! mkdir app

In [11]:
%%writefile app/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
# uvloop==0.15.2
# fastapi-utils
google-cloud-aiplatform
merlin-models
nvtabular
gcsfs
google-cloud-storage

Writing app/requirements.txt


In [12]:
%%writefile app/dataset_to_tensors.py

try:
    import cudf
except ImportError:
    cudf = None
import pandas as pd
import tensorflow as tf
from typing import Dict
from merlin.io import Dataset
import itertools


def cupy_array_to_tensor(array):
    return tf.experimental.dlpack.from_dlpack(array.reshape(-1, 1).toDlpack())

def numpy_array_to_tensor(array):
    return tf.convert_to_tensor(array.reshape(-1, 1))

def cudf_series_to_tensor(col) -> tf.Tensor:
    "Convert a cudf.Series to a TensorFlow Tensor with DLPack"
    if isinstance(col.dtype, cudf.ListDtype):
        values = col.list.leaves.values
        offsets = col.list._column.offsets.values
        row_lengths = offsets[1:] - offsets[:-1]
        return cupy_array_to_tensor(values), cupy_array_to_tensor(row_lengths)
    else:
        return cupy_array_to_tensor(col.values)

def pandas_series_to_tensor(col) -> tf.Tensor:
    if len(col) and pd.api.types.is_list_like(col.values[0]):
        values = pd.Series(itertools.chain(*col)).values
        row_lengths = col.map(len).values
        return numpy_array_to_tensor(values), numpy_array_to_tensor(row_lengths)
    else:
        return numpy_array_to_tensor(col.values)
        
    
def dataset_to_tensors(dataset: Dataset) -> Dict[str, tf.Tensor]:
    """Convert a DataFrame to Dict of Tensors"""
    df = dataset.to_ddf().compute()
    if isinstance(df, pd.DataFrame):
        col_to_tensor = pandas_series_to_tensor
    else:
        col_to_tensor = cudf_series_to_tensor
    return {
        column: col_to_tensor(df[column])
        for column in df.columns
    }

Writing app/dataset_to_tensors.py


In [13]:
%%writefile app/predictor.py
import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf
import time
import logging
from dataset_to_tensors import *


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = [
    'collaborative', 
    'album_name_pl', 
    'artist_genres_pl', 
    'artist_name_pl', 
    'artist_pop_can', 
    'description_pl', 
    'duration_ms_songs_pl', 
    'n_songs_pl', 
    'name', 
    'num_albums_pl', 
    'num_artists_pl', 
    'track_name_pl', 
    'track_pop_pl', 
    'duration_ms_seed_pl', 
    'pid', 
    'track_uri_pl'
]

float_num_fix = ['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
float_list_fix = ['track_pop_pl', 'duration_ms_songs_pl']
    
def fix_list_num_dtypes(num_list):
    "this fixes lists of ints to list of floats converted in json input"
    return [float(x) for x in num_list]

def fix_num_dtypes(num):
    "this fixes ints and casts to floats"
    return float(num)

def fix_types(k, v):
    if k in float_num_fix:
        return fix_num_dtypes(v)
    if k in float_list_fix:
        return fix_list_num_dtypes(v)
    else:
        return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs) == list:
        header = inputs[0]
        reordered_header_dict = {k: fix_types(k,header[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T
        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: fix_types(k,ti[k]) for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        reordered_dict = {k: fix_types(k,inputs[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_dict, orient='index').T
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        logging.info("loading model and workflow")
        start_init = time.process_time()
        
        #test_bucket = 'gs://jt-merlin-scaling'
        # self.model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query_model_merlin" ))
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow")) # TODO: parameterize
        self.model = tf.keras.models.load_model(artifacts_uri)
        
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow"))
        # self.workflow = nvt.Workflow.load('gs://jt-merlin-scaling/nvt-last5-v1full/nvt-analyzed') # TODO: parametrize
        self.workflow = nvt.Workflow.load("/docker_workflow/workflow")
        
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl'
            ]
        )
        return self
        
    def predict(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        self.n_rows = len(prediction_input)
        start = time.process_time()
        pandas_instance = create_pandas_instance(prediction_input[0])
        #logging.info(f"Pandas conversion took {time.process_time() - start} seconds")
        print(f"Pandas conversion took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_inputs = nvt.Dataset(pandas_instance)
        #logging.info(f"NVT data loading took {time.process_time() - start} seconds")
        print(f"NVT data loading took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_instance = self.workflow.transform(transformed_inputs)
        print(f"Workflow transformation took {time.process_time() - start} seconds")

        # def predict(self, instances):
        start = time.process_time()
        
        batch = dataset_to_tensors(transformed_instance)
        print(f"converting to dict_tensors took {time.process_time() - start} seconds")
        start = time.process_time()
        output = self.model(batch)
        print(f"Generating query embeddings took {time.process_time() - start} seconds")
        return transformed_instance, output, batch

Writing app/predictor.py


In [14]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os
import logging


from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    outputs = loaded_predictor.predict(instances)

    return {"predictions": outputs[1].numpy().tolist()}

Writing app/main.py


In [15]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing app/prestart.sh


In [16]:
%%writefile app/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Writing app/instances.json


In [17]:
#make it a package
!touch app/__init__.py

In [20]:
WORKFLOW_DIR='gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow'

!gsutil cp -r $WORKFLOW_DIR local_workflow

Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow/categories/unique.album_name_pl.parquet...
Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow/categories/unique.artist_genres_can.parquet...
Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow/categories/unique.artist_genres_pl.parquet...
Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow/categories/unique.artist_name_can.parquet...
- [4 files][ 23.6 MiB/ 23.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflow/categories/unique.artist_name_pl.parquet...
Copying gs://jt-merlin-scaling/nvt-last5-latest-12/nvt-processed/workflo

In [22]:
# !mkdir local_workflow
!mkdir local_workflow_jw

mkdir: cannot create directory ‘local_workflow_jw’: File exists


In [ ]:
# ! rm -rf {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{SERVING_DOCKERNAME}

In [24]:
# workflow_uri = 'gs://spotify-beam-v3/merlin-processed/workflow/2t-spotify-workflow'

# !gsutil cp -r -m $workflow_uri local_workflow_jw

In [26]:
%%writefile Dockerfile

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
WORKDIR /app 

COPY ./app/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt
#DEBUG CHANGES!!
RUN mkdir /docker_workflow
# RUN mkdir /docker_model
# ADD local_model /docker_model
ADD local_workflow /docker_workflow
#END DEBUG!


COPY ./app /app
EXPOSE 80
    
CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Writing Dockerfile


In [27]:
SERVER_IMAGE = "mm-prediction-cpr-jt"  # @param {type:"string"} 
REMOTE_IMAGE_NAME=f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{SERVER_IMAGE}"

!docker build -t $REMOTE_IMAGE_NAME .
# !gcloud builds submit -t $REMOTE_IMAGE_NAME .

Sending build context to Docker daemon  5.692GB
Step 1/9 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
22.12: Pulling from nvidia/merlin/merlin-tensorflow

d16dc43b: Already exists 
6ecee9ab: Already exists 
e0981273: Already exists 
03fa67ca: Already exists 
7f8e4744: Already exists 
73502f86: Already exists 
cf4cbe13: Already exists 
d495b912: Already exists 
6cd12433: Already exists 
3d945bde: Already exists 
5d54675a: Already exists 
25844090: Already exists 
a25c21ff: Already exists 
b700ef54: Already exists 
21e94f24: Already exists 
1b6fd612: Already exists 
e39433f4: Already exists 
1cba559e: Already exists 
054fc370: Already exists 
c99b1680: Already exists 
73498bf6: Already exists 
fd996372: Already exists 
c9ed50ff: Already exists 
db3dcf22: Already exists 
a7f6f15d: Already exists 
ed83fb5b: Already exists 
f4c1fb4d: Already exists 
fde98e21: Already exists 
e75d70d4: Already exists 
c32a822c: Already exists 
8a0bb031: Already exists 
f50cb482: Already exists 
3e86e

In [28]:
docker run -p 80:8080 \
            --name=mm-prediction-cpr-jt \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-beam-v3/merlin-processed/workflow/query-tower-model-22-12 \
            us-central1-docker.pkg.dev/hybrid-vertex/sp-mm-cpr/mm-prediction-cpr-jt

In [30]:
! curl localhost/health

curl: (7) Failed to connect to localhost port 80: Connection refused


In [32]:
! docker stop $SERVER_IMAGE
! docker rm $SERVER_IMAGE

mm-prediction-cpr-jt
mm-prediction-cpr-jt


In [33]:
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/sp-mm-cpr/mm-prediction-cpr-jt]

16900418: Preparing 
fa907c00: Preparing 
86ab44ad: Preparing 
ac39c605: Preparing 
087c6cbb: Preparing 
b5b3875a: Preparing 
2e747ad7: Preparing 
46caf956: Preparing 
c4f4a731: Preparing 
6f8ac904: Preparing 
55970ccc: Preparing 
1e82e910: Preparing 
7c23e98f: Preparing 
70ded861: Preparing 
da794091: Preparing 
493f5d44: Preparing 
caa9de82: Preparing 
03ad2a31: Preparing 
5e1b9e48: Preparing 
f4addbfc: Preparing 
81e7ae87: Preparing 
3d5ddcb4: Preparing 
a992b719: Preparing 
f10e020a: Preparing 
7f70fff9: Preparing 
2562fda1: Preparing 
decbb6f6: Preparing 
1cea2b76: Preparing 
fcaf77a9: Preparing 
e31ca3c7: Preparing 
f58ddb35: Preparing 
61786022: Preparing 
1f7b3d75: Preparing 
e5c17b0b: Preparing 
15f32335: Preparing 
fb54d10a: Preparing 
90be1950: Preparing 
84fa095a: Preparing 
cab8959e: Preparing 
7d764727: Preparing 
d4d05916: Preparing 
34553782

In [34]:
MODEL_DEPLOY_VERSION = "v23"

EXPERIMENT_RUN_DIR='gs://jt-merlin-scaling/new9-2tower-merlin-tf-jtv24/run-20230228-170848'
MODEL_DIR=f'{EXPERIMENT_RUN_DIR}/model_dir'

print(f"EXPERIMENT_RUN_DIR: {EXPERIMENT_RUN_DIR}")
print(f"MODEL_DIR: {MODEL_DIR}")


MODEL_DISPLAY_NAME=f"mm-qtower-{MODEL_DEPLOY_VERSION}"
MODEL_ARTIFACT_URI = f'{MODEL_DIR}/query_tower'

print(f"MODEL_ARTIFACT_URI: {MODEL_ARTIFACT_URI}")

EXPERIMENT_RUN_DIR: gs://jt-merlin-scaling/new9-2tower-merlin-tf-jtv24/run-20230228-170848
MODEL_DIR: gs://jt-merlin-scaling/new9-2tower-merlin-tf-jtv24/run-20230228-170848/model_dir
MODEL_ARTIFACT_URI: gs://jt-merlin-scaling/new9-2tower-merlin-tf-jtv24/run-20230228-170848/model_dir/query_tower


In [39]:
REMOTE_IMAGE_NAME

'us-central1-docker.pkg.dev/hybrid-vertex/sp-mm-cpr/mm-prediction-cpr-jt'

In [35]:
start = time.process_time()

model = vertex_ai.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=MODEL_ARTIFACT_URI,
        serving_container_image_uri=REMOTE_IMAGE_NAME,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args='--gpus all',
        # parent_model=PARENT_MODEL,
        sync=True,
    )

print(f"Created model in {round((time.process_time() - start),2)} seconds\n")

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/3931415975299121152/operations/4295169399981080576
Model created. Resource name: projects/934903580331/locations/us-central1/models/3931415975299121152@1
To use this Model in another session:
model = aiplatform.Model('projects/934903580331/locations/us-central1/models/3931415975299121152@1')
Created model in 0.15 seconds



In [36]:
# service account for predictions
VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'

# model and endpoint display names
# MODEL_DISPLAY_NAME=f"mm-retrieval-{VERSION_serving}"
DEPLOYED_MODEL_DISPLAY_NAME=f"nb-deployed-{MODEL_DISPLAY_NAME}"
ENDPOINT_DISPLAY_NAME = f"mm-endpoint-{MODEL_DEPLOY_VERSION}"

# Endpoint resource config
# DEPLOY_COMPUTE="n1-standard-4"
# DEPLOY_GPU="NVIDIA_TESLA_T4"
# DEPLOY_NGPU=1
# MIN_NODES=1
# MAX_NODES=1
# TRAFFIC=100
traffic_percentage = 100
machine_type = "n1-highmem-16"
accelerator_type = "NVIDIA_TESLA_T4"
accelerator_count = 1
min_replica_count = 1
max_replica_count = 1

print(f"DEPLOYED_MODEL_DISPLAY_NAME: {DEPLOYED_MODEL_DISPLAY_NAME}")
print(f"ENDPOINT_DISPLAY_NAME: {ENDPOINT_DISPLAY_NAME}")

DEPLOYED_MODEL_DISPLAY_NAME: nb-deployed-mm-qtower-v23
ENDPOINT_DISPLAY_NAME: mm-endpoint-v23


In [37]:
start = time.process_time()

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=LOCATION,
)

print(f"Created endpoint in {round((time.process_time() - start),2)} seconds\n")

print(endpoint)

Creating Endpoint
Create Endpoint backing LRO: projects/934903580331/locations/us-central1/endpoints/7280926416714072064/operations/7463451737836224512
Endpoint created. Resource name: projects/934903580331/locations/us-central1/endpoints/7280926416714072064
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/7280926416714072064')
Created endpoint in 0.04 seconds

resource name: projects/934903580331/locations/us-central1/endpoints/7280926416714072064


In [38]:
start = time.process_time()

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_DISPLAY_NAME,
    machine_type=machine_type,
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    traffic_percentage=traffic_percentage,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    service_account=VERTEX_SA,
    sync=True,
    # deploy_request_timeout=1800
)

print(f"Deployed model to endpoint in {round((time.process_time() - start),2)} seconds\n")

print(model)

Deploying model to Endpoint : projects/934903580331/locations/us-central1/endpoints/7280926416714072064
Deploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/7280926416714072064/operations/8438481057161936896
Endpoint model deployed. Resource name: projects/934903580331/locations/us-central1/endpoints/7280926416714072064
Deployed model to endpoint in 0.48 seconds

resource name: projects/934903580331/locations/us-central1/models/3931415975299121152


In [40]:
print(endpoint.gca_resource.deployed_models[0])

id: "2380288742507151360"
model: "projects/934903580331/locations/us-central1/models/3931415975299121152"
display_name: "nb-deployed-mm-qtower-v23"
create_time {
  seconds: 1678145739
  nanos: 478510000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-highmem-16"
    accelerator_type: NVIDIA_TESLA_T4
    accelerator_count: 1
  }
  min_replica_count: 1
  max_replica_count: 1
}
service_account: "934903580331-compute@developer.gserviceaccount.com"
model_version_id: "1"



In [41]:
TEST_INSTANCE = {
    'collaborative': 'false',
    'album_name_pl': [
        "There's Really A Wolf", 'Late Nights: The Album','American Teen', 'Crazy In Love', 'Pony'
    ], 
    'artist_genres_pl': [
        "'hawaiian hip hop', 'rap'",
       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"
    ], 
    'artist_name_pl': [
        'Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9','William Singe'
    ], 
    'artist_pop_can': 82.0, 
    'description_pl': '', 
    'duration_ms_songs_pl': [
        237506.0, 217200.0, 219080.0, 226400.0, 121739.0
    ], 
    'n_songs_pl': 8.0, 
    'name': 'Lit Tunes ', 
    'num_albums_pl': 8.0, 
    'num_artists_pl': 8.0, 
    'track_name_pl': [
        'Losin Control', 'Paradise', 'Location','Crazy In Love - Remix', 'Pony'
    ], 
    'track_pop_pl': [
        79.0, 58.0, 83.0, 71.0, 57.0
    ],
    'duration_ms_seed_pl': 51023.1,
    'pid': 1,
    'track_uri_pl': [
        'spotify:track:4cxMGhkinTocPSVVKWIw0d',
        'spotify:track:1wNEBPo3nsbGCZRryI832I',
        'spotify:track:152lZdxL1OR0ZMW6KquMif',
        'spotify:track:2f4IuijXLxYOeBncS60GUD',
        'spotify:track:4Lj8paMFwyKTGfILLELVxt'
    ]
}

In [42]:
start = time.process_time()

playlist_emb = endpoint.predict(instances=[TEST_INSTANCE])

print(f"query conversion: {round((time.process_time() - start),4)} seconds")
print(f"Vector Dimensions: {len(playlist_emb.predictions[0])}\n")

print(f"embeddings: {playlist_emb.predictions}")

query conversion: 0.0824 seconds
Vector Dimensions: 128

embeddings: [[0.1575801819562912, 0.0250331275165081, 0.0, 0.01443512085825205, 0.004722597077488899, 0.0, 0.06282097101211548, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007788579910993576, 0.0, 0.01886783353984356, 0.0, 0.01729366183280945, 0.0, 0.006211482454091311, 0.00838406290858984, 0.03776045143604279, 0.02149274945259094, 0.0, 0.0, 0.03309539705514908, 0.04335452243685722, 0.1061182543635368, 0.0, 0.001437801984138787, 0.002923614345490932, 0.08064340800046921, 0.0, 0.0, 0.0, 0.0839158147573471, 0.0, 0.007533694617450237, 0.0, 0.08348245173692703, 0.0, 0.0, 0.0, 0.06940964609384537, 0.0, 0.0, 0.0, 0.05281869694590569, 0.0, 0.0, 0.09006726741790771, 0.0, 0.02304030582308769, 0.0, 0.0, 0.1235332861542702, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05732613429427147, 0.1368753612041473, 0.0, 0.0, 0.04231433570384979, 0.02258080057799816, 0.1451370418071747, 0.0, 0.0, 4.405411891639233e-05, 0.01958194933831692, 0.08558918535709381, 0.028923241421

In [43]:
start = time.process_time()

playlist_emb = endpoint.predict(instances=[TEST_INSTANCE])

end = time.process_time()

total_train_time = round((end - start),4)

print(f"query conversion: {total_train_time}")

query conversion: 0.0261
